In [ ]:
import numpy as np
from BPmodule.BPGen import CreateData
from BPnumba.GeneticOperators import CreatePoblation,InstancePob,CreateHeuristicPob
from BPnumba.AGH import createAG
import matplotlib.pyplot as plt
from numba.typed import List as NumbaList


In [ ]:
contenedor,data = CreateData(algorithm=2,problem=2)


In [ ]:
datos = NumbaList(data[0])

In [ ]:
sm = 0
for dat in datos:
    sm += np.prod(dat)
sm,np.prod(contenedor)

In [ ]:
idBoxes = np.arange(1,len(datos)+1,dtype=np.int64)
n= len(datos)
pop= 50
initial = CreateHeuristicPob(pop,datos,contenedor)
maxItr=1000
n,pop

In [ ]:
print(initial[0])

In [7]:
Pob = InstancePob(NumbaList(initial),datos,contenedor)

In [8]:
Pob[0].genome

ListType[int64]([1, 9, 21, 3, 2, 20, 8, 19, 4, 18, 16, 17, 22, 24, 12, 7, 14, 15, 5, 10, 23, 13, 25, 6, 11, ...])

In [ ]:
ag = createAG(ps=0.85,pc=0.75,adaptive=True)

In [ ]:
ag.Train(maxItr,Pob,datos,contenedor)

In [ ]:
ag.BestInd.fi,ag.BestInd.codeSolution

In [ ]:
n-ag.BestInd.load

In [ ]:
plt.figure()
plt.plot(ag.bestfi)
plt.ylim(0,1)
plt.show()